[download this notebook here](https://github.com/HumanCompatibleAI/imitation/blob/master/docs/tutorials/1_train_bc.ipynb)
# Train an Agent using Behavior Cloning

Behavior cloning is the most naive approach to imitation learning. 
We take the transitions of trajectories taken by some expert and use them as training samples to train a new policy.
The method has many drawbacks and often does not work. 
However in this example, where we use an agent for the seals/CartPole-v0 environment, it is feasible.

Note that we use a variant of the CartPole environment from the seals package, which has fixed episode durations. Read more about why we do this [here](https://imitation.readthedocs.io/en/latest/main-concepts/variable_horizon.html).

First we need some kind of expert in CartPole so we can sample some expert trajectories.
For convenience we just download one from the HuggingFace model hub.

If you want to train an expert yourself have a look at the [training documenation](https://rl-baselines3-zoo.readthedocs.io/en/master/guide/train.html#basic-usage) of RL Baselines3 Zoo.

In [11]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper

In [12]:
env = make_vec_env(
    "seals:seals/HalfCheetah-v1",
    rng=np.random.default_rng(),
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals/HalfCheetah-v1",
    venv=env,
)

/Users/shiva/anaconda3/envs/irlproj/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/Users/shiva/anaconda3/envs/irlproj/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/Users/shiva/anaconda3/envs/irlproj/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


Let's quickly check if the expert is any good.
We usually should be able to reach a reward of 500, which is the maximum achievable value.

In [20]:
trajectories = 10
epochs = 1
eval_episodes = 10

In [21]:
from stable_baselines3.common.evaluation import evaluate_policy

reward, _ = evaluate_policy(expert, env, n_eval_episodes=eval_episodes)
print(reward)

1667.8850105000001


Now we can use the expert to sample some trajectories.
We flatten them right away since we are only interested in the individual transitions for behavior cloning.
`imitation` comes with a number of helper functions that makes collecting those transitions really easy. First we collect 50 episode rollouts, then we flatten them to just the transitions that we need for training.

Note that the rollout function requires a vectorized environment and needs the `RolloutInfoWrapper` around each of the environments. This is why we passed the `post_wrappers` argument to `make_vec_env` above.

In [22]:
from imitation.data import rollout

rng = np.random.default_rng()
rollouts = rollout.rollout(
    expert,
    env,
    rollout.make_sample_until(min_timesteps=None, min_episodes=trajectories),
    rng=rng,
)
transitions = rollout.flatten_trajectories(rollouts)

Let's have a quick look at what we just generated using those library functions:

In [23]:
print(
    f"""The `rollout` function generated a list of {len(rollouts)} {type(rollouts[0])}.
After flattening, this list is turned into a {type(transitions)} object containing {len(transitions)} transitions.
The transitions object contains arrays for: {', '.join(transitions.__dict__.keys())}."
"""
)
# print("full", rollouts[0])
# print("obs", rollouts[0].obs.shape)
# print("acts", rollouts[0].acts.shape)


The `rollout` function generated a list of 16 <class 'imitation.data.types.TrajectoryWithRew'>.
After flattening, this list is turned into a <class 'imitation.data.types.Transitions'> object containing 16000 transitions.
The transitions object contains arrays for: obs, acts, infos, next_obs, dones."



After we collected our transitions, it's time to set up our behavior cloning algorithm.

In [24]:
from imitation.algorithms import bc

bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

As you can see the untrained policy only gets poor rewards:

In [25]:
reward_before_training, _ = evaluate_policy(bc_trainer.policy, env, eval_episodes)
print(f"Reward before training: {reward_before_training}")

Reward before training: 0.18239499999999997


After training, we can match the rewards of the expert (500):

In [26]:
bc_trainer.train(n_epochs=epochs)
reward_after_training, _ = evaluate_policy(bc_trainer.policy, env, eval_episodes)
print(f"Reward after training: {reward_after_training}")

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00851 |
|    entropy        | 8.51     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 101      |
|    loss           | 7.28     |
|    neglogp        | 7.29     |
|    prob_true_act  | 0.000727 |
|    samples_so_far | 32       |
--------------------------------


500batch [00:00, 685.01batch/s]


Reward after training: 738.9631741000001
